In [ ]:
rm(list = ls())
setwd(rprojroot::find_rstudio_root_file())
source("base/Preprocess_data.R")

In [ ]:
a = grid_pred(c("SOI","PDO","MEIv2","ONI","ESPI"),-1,"mean")#
c = grid_pred(c("STORAGE"),1,"last")

predictors = c(a,c)
predictors

In [ ]:
#all available catchments, no data 6008005, 7317005, 7355002, 8106001
catchments_attributes_filename = "data_input/attributes/attributes_49catchments_ChileCentral.csv" 
cod_cuencas = read.csv(catchments_attributes_filename)$cod_cuenca [-c(32,40,45,49)]
cod_cuencas

In [ ]:
catchment_code = "4703002"

In [ ]:
#input data from preprocess
data_input <- preprocess_data(datetime_initialisation = '2022-11-01',
                              forecast_mode = "cv",
                              catchment_code = catchment_code,
                              predictor_list = predictors)
data_input  %>% names

In [ ]:
# regression model parameters
source("base/Regression_model.R")
library('caret')
X_train = data_input$X_train
y_train = data_input$y_train$volume
resampling_method = 'none'
preProcess = c("center", "scale")
method = 'lm'
trcontrol = trainControl(method = resampling_method, savePredictions = "all")
X_train  %>% names
data_input$info$predictor_list

In [ ]:
# remove correlated predictors
cor_matrix <- cor(X_train)
high_cor   <- findCorrelation(cor_matrix, cutoff = 0.8, verbose = F,exact = T)
X_train_uncorr = X_train[, -high_cor]
X_train_uncorr  %>% names

In [ ]:
predictors2 = data_input$info$predictor_list[[1]][-high_cor]
predictors2

In [ ]:
corrplot::corrplot(cor(X_train_uncorr,y_train))
corrplot::corrplot(cor_matrix,type = 'upper',diag = F)

In [ ]:
predictors_comb = do.call("c", lapply(seq_along(predictors2), function(i) c(combn(predictors2, i, FUN = list))))
predictors_comb

In [ ]:
#months of initialisation
months_initialisation = seq(5,9)
months_initialisation

In [ ]:
p1 = months_initialisation
p2 = predictors_comb



list_lengths <- function(...){
  list_lengths <- lapply(list(...), length)
  total_iter <- prod(unlist(list_lengths))
  return(total_iter)
}


max_iter = list_lengths(p1,p2)
max_iter

In [ ]:
preProcess

In [ ]:

i = 1
a = list()

for(month_initialisation in p1){
for(predictor in p2){

#input data from preprocess
data_input <- preprocess_data(datetime_initialisation = lubridate::make_date(2022,month_initialisation,1),
                              forecast_mode = "cv",
                              catchment_code = catchment_code,
                              predictor_list = predictor)
      #model

    regression_model = 
train(
  data_input$X_train,
  data_input$y_train$volume,
  metric = "RMSE",
  trControl = trcontrol,
  method = method,
  preProcess = preProcess
)
    
    
    a[[i]] = regression_model
    i = i+1

}
}


In [ ]:
# #plot predictant histogram
# library(ggplot2)
# ggplot(data = (data_input$y_train))+geom_histogram(aes(x = volume),bins = 5)

In [ ]:
# # determine variable importance
# index_imp = which(imp$lmg > 0.1)
# index_imp
# # remove predictors with low importance
# cols = colnames(X_train_uncorr)

# x_reduced <- data.frame(X_train_uncorr[,index_imp])
# colnames(x_reduced) = cols[index_imp]
# x_reduced  %>% names